In [1]:
#%pip install git+https://github.com/royerlab/tracksdata
%pip uninstall -y tracksdata
%pip install git+https://github.com/yfukai/tracksdata

Found existing installation: tracksdata 0.1.dev639+ge849db006
Uninstalling tracksdata-0.1.dev639+ge849db006:
  Successfully uninstalled tracksdata-0.1.dev639+ge849db006
Note: you may need to restart the kernel to use updated packages.
  Cloning https://github.com/yfukai/tracksdata to /private/var/folders/81/kh0zpmbd00g39vwz69lkk4mc0000gn/T/pip-req-build-ukxhjx8j
  Running command git clone --filter=blob:none --quiet https://github.com/yfukai/tracksdata /private/var/folders/81/kh0zpmbd00g39vwz69lkk4mc0000gn/T/pip-req-build-ukxhjx8j
  Resolved https://github.com/yfukai/tracksdata to commit e849db006dfcff170e5d98a786376307bdf129a8
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for tracksdata: filename=tracksdata-0.1.dev639+ge849db006-py3-none-any.whl size=200773 sha256=4edacb9a0af63d59a0863e97232174de2d785e48f11402a506c4af10e53a5277
  Stored in directory: /private/var/folders/81/kh0zpmb

In [2]:
image_path = "/Volumes/work/fukai/241203-cell-picker/aligned_image.zarr"
graph_path = "/Users/fukai/Downloads/aligned_labels_tracked4.db"

In [3]:
import tracksdata as td
import os
os.environ["NAPARI_ASYNC"] = "1"
import zarr
from dask import array as da

In [4]:
image = da.from_zarr(zarr.open(image_path, mode="r"))
graph = td.graph.SQLGraph(drivername="sqlite", database=graph_path)

In [11]:
image = da.zeros_like(image)

In [5]:
import napari_travali2 as travali2
travali2._stateful_widget.find_track_successors(graph,885,"label")

source_id,label
i64,i64
107000000438,4497
107000000438,4498


In [6]:
graph.filter(node_ids=[884]).node_attrs()

node_id,t,mask,bbox,label,y,x,area,termination_annotation,verified,tracklet_id
i64,i64,object,list[i64],i64,f64,f64,f64,null,i64,i64
884,0,"Mask(bbox=[11477:11515, 6009:6045])","[11477, 6009, … 6045]",885,11495.576744,6026.60186,1075.0,null,0,-1


In [7]:
graph.successors(node_ids=[107000000438], return_attrs=True)

{107000000438: shape: (2, 12)
 ┌─────────────┬─────────────┬─────┬─────────────┬───┬───────┬─────────────┬──────────┬─────────────┐
 │ source_id   ┆ node_id     ┆ t   ┆ mask        ┆ … ┆ area  ┆ termination ┆ verified ┆ tracklet_id │
 │ ---         ┆ ---         ┆ --- ┆ ---         ┆   ┆ ---   ┆ _annotation ┆ ---      ┆ ---         │
 │ i64         ┆ i64         ┆ i64 ┆ object      ┆   ┆ f64   ┆ ---         ┆ bool     ┆ i64         │
 │             ┆             ┆     ┆             ┆   ┆       ┆ null        ┆          ┆             │
 ╞═════════════╪═════════════╪═════╪═════════════╪═══╪═══════╪═════════════╪══════════╪═════════════╡
 │ 10700000043 ┆ 10800000191 ┆ 108 ┆ Mask(bbox=[ ┆ … ┆ 468.0 ┆ null        ┆ false    ┆ -1          │
 │ 8           ┆ 2           ┆     ┆ 11480:11507 ┆   ┆       ┆             ┆          ┆             │
 │             ┆             ┆     ┆ , 6032:6…   ┆   ┆       ┆             ┆          ┆             │
 │ 10700000043 ┆ 10800000191 ┆ 108 ┆ Mask(bbox=[ ┆ …

In [8]:
from napari import Viewer
import napari_travali2 as travali2
from napari_travali2._logging import logger
from napari_travali2.actionable_tracks import actionable_tracks as at
import importlib
importlib.reload(travali2)
importlib.reload(travali2._stateful_widget)
import numpy as np

In [9]:
tracks = at.ActionableTracks(graph, tracklet_id_attr_name="label")
gav = td.array.GraphArrayView(tracks.graph, shape=tuple([image.shape[0],*image.shape[2:]]), attr_key="label")

In [ ]:
logger.setLevel("DEBUG")
logger.info("Starting napari-travali2")

#tracks = at.ActionableTracks(graph, tracklet_id_attr_name="label")
#gav = td.array.GraphArrayView(tracks.graph, shape=tuple([image.shape[0],*image.shape[2:]]), attr_key="label")

viewer = Viewer()

widget = travali2.StateMachineWidget(viewer, tracks, image[:,0,:,:], 
                            verified_track_ids=[],
                            candidate_track_ids=[],
                            crop_size=2048,
                            gav=gav)
viewer.window.add_dock_widget(widget, area="right")
viewer.dims.set_current_step(0,0)

/Users/fukai/projects/napari-travali2/.venv/lib/python3.12/site-packages/napari/_vispy/layers/scalar_field.py:197: UserWarning: data shape (20805, 14617) exceeds GL_MAX_TEXTURE_SIZE 16384 in at least one axis and will be downsampled. Rendering is currently in 2D mode.
  warnings.warn(
/Users/fukai/projects/napari-travali2/.venv/lib/python3.12/site-packages/napari/_vispy/layers/scalar_field.py:197: UserWarning: data shape (20805, 14617) exceeds GL_MAX_TEXTURE_SIZE 16384 in at least one axis and will be downsampled. Rendering is currently in 2D mode.
  warnings.warn(
/Users/fukai/projects/napari-travali2/.venv/lib/python3.12/site-packages/napari/_vispy/layers/scalar_field.py:197: UserWarning: data shape (20805, 14617) exceeds GL_MAX_TEXTURE_SIZE 16384 in at least one axis and will be downsampled. Rendering is currently in 2D mode.
  warnings.warn(
/Users/fukai/projects/napari-travali2/.venv/lib/python3.12/site-packages/napari/_vispy/layers/scalar_field.py:197: UserWarning: data shape (20

In [11]:
mask = td.nodes.Mask(mask=np.ones([23, 23], dtype=bool), 
                     bbox=[11890, 3985, 11913, 4008])
attrs = {
    'mask': mask, 
    'bbox': mask.bbox, 
    't': 1, 
    'termination_annotation': '', 
         'y': np.float64(11900.86231884058), 
         'x': np.float64(3996.222222222),
         'area': np.float64(414.0), 
         'verified': False, 
         'tracklet_id': -1, 
         'label': 925
}
tracks.graph.add_node(attrs=attrs, validate_keys=False)


1000001484